# Robot in a room
We implement a determinsitic enviornment simulation for a robot in a room problem using basic $Q$ learning with the modified bellman equation given as
$$
Q_{t+1}(s,a) = Q_{t}(s,a) + \alpha\left(R(s,a) + \gamma\max_{a^{\prime}}\{Q_{t}(s',a')\} - Q_{t}(s,a)\right)
$$
where $a : s \mapsto s'$.

In [25]:
%pip install numpy
import numpy as np

Note: you may need to restart the kernel to use updated packages.


This is how our enviornment looks

|  C  | F | I | L |
| -- | -- | -- | -- |
| B | X (E) | H | K |
| A | D | G | J |

-

|  (0,2)  |  (1,2)  | (2,2) | (3,2) |
| -- | -- | -- | -- |
| (0,1) | X | (2,1) | (3,1) |
| (0,0) | (1,0) | (2,0) | (3,0) |


## Stochasticity?
$$
Q_{t+1}(s,a) = Q_{t}(s,a) + \alpha\left(R(s,a) + \gamma\sum_{s^{\prime}}P_{s,a}(s^{\prime})\max_{a^{\prime}}\{Q_{t}(s',a')\} - Q_{t}(s,a)\right)
$$

In [26]:
def down(a):
    return (a[0],a[1] - 1)
def left(a):
    return (a[0] - 1,a[1])
def right(a):
    return (a[0] + 1,a[1])
def up(a):
    return (a[0],a[1] + 1)

ACTIONS = [up, down, left, right]

def valid_neighbours(a,w,h,blocks):
    res = []
    for act in range(len(ACTIONS)):
        j = ACTIONS[act](a)
        if j[0] < 0 or j[0] >= w or j[1] < 0 or j[1] >= h or j in blocks:
            continue
        res.append(j)
    return res

In [48]:
WIDTH = 4
HEIGHT = 3
WORST_REWARD = -999
REWARD_TO_STAY = -0.1
REWARD_TO_COMPLETE = 10

STATES = []
SYMBOL = {}
symbol = 'A'

BLOCKS = [(1,1)]


for w in range(WIDTH):
    for h in range(HEIGHT):
        if (w,h) in BLOCKS:
            symbol = chr(ord(symbol) + 1)
            continue
        STATES.append((w,h))
        SYMBOL[(w,h)] = symbol
        symbol = chr(ord(symbol) + 1)


REWARDS = {}
Q = {}
for i in STATES:
    REWARDS[i] = [WORST_REWARD]*4
    Q[i] = [0]*4
    for act in range(len(ACTIONS)):
        j = ACTIONS[act](i)
        if j[0] < 0 or j[0] >= WIDTH or j[1] < 0 or j[1] >= HEIGHT or j in BLOCKS:
            continue
        REWARDS[i][act] = REWARD_TO_STAY

In [49]:
class RoomBot:
    def __init__(self, states = STATES, q = Q, rewards = REWARDS, blocks = BLOCKS, width=WIDTH, height=HEIGHT, learning_rate=0.2, discount_rate=0.9):
        # hyperparameters
        self.learning_rate = learning_rate
        self.discount_rate = discount_rate
        
        self.width = width
        self.height = height
        self.rewards = rewards
        self.states = states
        self.blocks = blocks
        self.q = q
        self.goal = None

    def setPriority(self, dest, val=WORST_REWARD):
        for act in range(len(ACTIONS)):
            j = ACTIONS[act](dest)
            if j[0] < 0 or j[0] >= self.width or j[1] < 0 or j[1] >= self.height or j in self.blocks:
                continue
            if act == 0 or act == 2:
                self.rewards[j][act+1] = val
            else:
                self.rewards[j][act-1] = val

    def train(self, goal, iterations = 100):
        self.goal = goal

        self.setPriority(goal, REWARD_TO_COMPLETE)

        for episode in range(iterations):
            if episode%10 == 0:
                print(f"Running episode {episode}")
            spawn_w = np.random.randint(0,self.width)
            spwan_h = np.random.randint(0,self.height)
            spawn = (spawn_w, spwan_h)
            if spawn in self.blocks:
                continue

            while spawn != goal:
                v_neighbour = valid_neighbours(spawn, self.width, self.height, self.blocks)
                action = 0
                if np.random.rand() < 0.4:
                    action = np.random.randint(0, 4)
                else:
                    action = self.q[spawn].index(max(self.q[spawn]))
            
                next = ACTIONS[action](spawn)
                if next not in v_neighbour:
                    continue

                temporal_difference = self.rewards[spawn][action] + self.discount_rate * max(self.q[next]) - self.q[spawn][action]
                self.q[spawn][action] += self.learning_rate*temporal_difference
                spawn = next
    
    def route(self, start):
        if self.goal is None:
            print("Please first train the model to learn about the enviorment with a goal")
            return
        
        path = [SYMBOL[start]]
        while start != self.goal:
            max_q = self.q[start][0]
            next = ACTIONS[0](start)
            for i in range(len(ACTIONS)):
                if max_q < self.q[start][i]:
                    max_q = self.q[start][i]
                    next = ACTIONS[i](start)
            start = next
            path.append(SYMBOL[start])

        return path

In [50]:
test = RoomBot(discount_rate=0.9999)
test.setPriority((2,1),0.1)
# test.setPriority((2,1),0.3)
test.setPriority((3,1))
test.setPriority((2,0))

# test.setPriority((3,0))



In [51]:
test.train((3,2),iterations=1000)

Running episode 0
Running episode 10
Running episode 20
Running episode 30
Running episode 40
Running episode 50
Running episode 60
Running episode 70
Running episode 80
Running episode 90
Running episode 100
Running episode 110
Running episode 120
Running episode 130
Running episode 140
Running episode 150
Running episode 160
Running episode 170
Running episode 180
Running episode 190
Running episode 200
Running episode 210
Running episode 220
Running episode 230
Running episode 240
Running episode 250
Running episode 260
Running episode 270
Running episode 280
Running episode 290
Running episode 300
Running episode 310
Running episode 320
Running episode 330
Running episode 340
Running episode 350
Running episode 360
Running episode 370
Running episode 380
Running episode 390
Running episode 400
Running episode 410
Running episode 420
Running episode 430
Running episode 440
Running episode 450
Running episode 460
Running episode 470
Running episode 480
Running episode 490
Running epi

In [52]:
print(test.route((0,0)))

['A', 'B', 'C', 'F', 'I', 'L']


This is how our enviornment looks

|  C  | F | I | L |
| -- | -- | -- | -- |
| B | X (E) | H | K |
| A | D | G | J |

-

|  (0,2)  |  (1,2)  | (2,2) | (3,2) |
| -- | -- | -- | -- |
| (0,1) | X | (2,1) | (3,1) |
| (0,0) | (1,0) | (2,0) | (3,0) |


In [46]:
x = test.q
for y in x:
    print(f"{SYMBOL[y]} : {x[y]}")

A : [25.06385224650635, 0, 0, 22.80164282911937]
B : [25.866429292365666, 21.221247969622805, 0, 0]
C : [0, 24.1713978419022, 0, 26.2872902016783]
D : [0, 0, 24.53579866659792, -974.2117007516008]
F : [0, 0, 25.648148631694525, 26.566683912466818]
G : [26.63405460562341, 0, 23.67110004287544, -0.05099999992259679]
H : [26.628460550219106, -972.5580279777276, 0, -972.5949781549751]
I : [0, 26.69876777463143, 26.390255133150603, 9.999999999999996]
J : [-973.3937269470966, 0, -973.0669561157966, 0]
K : [9.999999999999996, -0.05099999953864089, 26.59637430178506, 0]
L : [0, 0, 0, 0]


In [47]:
x = test.rewards
for y in x:
    print(f"{SYMBOL[y]} : {x[y]}")

A : [-0.051, -999, -999, -0.051]
B : [-0.051, -0.051, -999, -999]
C : [-999, -0.051, -999, -0.051]
D : [-999, -999, -0.051, -999]
F : [-999, -999, -0.051, -0.051]
G : [0.1, -999, -0.051, -0.051]
H : [-0.051, -999, -999, -999]
I : [-999, 0.1, -0.051, 10]
J : [-999, -999, -999, -999]
K : [10, -0.051, 0.1, -999]
L : [-999, -999, -0.051, -999]
